In [2]:
import numpy as np
import plotly.graph_objs as go
from ipywidgets import widgets
import scipy.stats as spst
 
 
lmbda_value=widgets.FloatSlider(
    value=3.5,
    min=0.1,
    max=30.0,
    step=0.1,
    description='lambda value',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)


RNG=widgets.Button(description="Generate RN")

search_rn=widgets.Checkbox(
    value=False,
    description='Search',
    disabled=False,
    indent=False
)


RN_text=widgets.Text(
    value="0.8",
    description='a value:',
    disabled=False
)

FB=widgets.HTML(
    value="<b></b>",
    placeholder='Some HTML',
    description='',
)
 

def process_data():
    lmbda=lmbda_value.value
    Outcomes=np.arange(0,lmbda*10)
    Probs=spst.poisson.pmf(Outcomes,lmbda)
    CDF= spst.poisson.cdf(Outcomes,lmbda)
    CDF=np.append(0,CDF)
    x=np.repeat(0,len(CDF))
      
    RN=np.fromstring(RN_text.value, dtype=float, sep=',')[0]
    return Probs, CDF,x,Outcomes,RN,lmbda

 

Probs,CDF,x,Outcomes,RN,lmbda=process_data()

 
# Assign an empty figure widget with two traces


def convert(Outcomes,CDF,RN):
    Noutcomes=len(Outcomes)
    for i in range(Noutcomes):
        if RN<=CDF[i+1]:
            return i

def convert_updown(lmbda,RN):
    #downward
    x=int(lmbda)
    downward=1
    if RN<spst.poisson.cdf(x,lmbda):
        x=x-1
        while RN<spst.poisson.cdf(x,lmbda):
            x=x-1
        x=x+1  
    else:
        downward=0
        x=x+1
        while RN>spst.poisson.cdf(x,lmbda):
            x=x+1
 
    return x,downward
    
        
i_ud,downward=convert_updown(lmbda,RN)            
    
    
     
        
i=convert(Outcomes,CDF,RN) 
#i_s=convert(Outcomes_s,CDF_s,RN)     

trace1=go.Bar(x=x,y=Probs, marker=dict(color=CDF,colorscale="PuBu"), 
              hoverinfo="text",text=Outcomes,name="Outcome regions"+' '*10   )
trace2=go.Scatter(x=[0],y=[RN],hoverinfo="text",text="u="+str(RN)+"<br>converts to "+str(Outcomes[i]),
                  marker=dict(size=15,color="red"),name="Random number")
trace3=go.Scatter(x=x,y=CDF,mode='markers+lines+text', hoverinfo="skip", text=np.round(CDF,2),
                  textposition="middle left",marker=dict(size=20,symbol="line-ew-open")
                  ,marker_color="black",name="CDF")
trace4=go.Scatter(x=x[1:i+1],y=CDF[1:i+1],mode='markers', hoverinfo="text",
                  textposition="middle left",marker=dict(size=9,symbol="x")
                  ,marker_color="blue",text="u>=cdf, move above",name="Failed comparisons")
trace5=go.Scatter(x=x[[i+1]],y=CDF[[i+1]],mode='markers', hoverinfo="text",
                  textposition="middle left",marker=dict(size=9,symbol="circle")
                  ,name="Successful comparison",marker_color="blue",text="u<cdf, stop <br> N_searches="+str(int(i)+1))

 
trace6=go.Scatter(x=x[[int(lmbda)+1]],y=CDF[[int(lmbda)+1]],mode='markers', hoverinfo="text",
                  textposition="middle left",marker=dict(size=9,symbol="star-triangle-up")
                  ,name="First comparison",marker_color="blue",text="u>=CDF(int(lmbda)), go upward")

trace7=go.Scatter(x=x[[int(lmbda)+1]],y=CDF[[int(lmbda)+1]],mode='markers', hoverinfo="text",
                  textposition="middle left",marker=dict(size=9,symbol="star-triangle-down")
                  ,name="First comparison",marker_color="blue",text="u<CDF(int(lmbda)), go downward")

trace8=go.Scatter(x=x[int(lmbda)+2:i_ud+1],y=CDF[int(lmbda)+2:i_ud+1],mode='markers', hoverinfo="text",
                  textposition="middle left",marker=dict(size=9,symbol="x")
                  ,marker_color="blue",text="u>=cdf, move above",name="Failed comparisons")
trace9=go.Scatter(x=x[i_ud+1:int(lmbda)+1],y=CDF[i_ud+1:int(lmbda)+1],mode='markers', hoverinfo="text",
                  textposition="middle left",marker=dict(size=9,symbol="x")
                  ,marker_color="blue",text="u<cdf, move below",name="Failed comparisons")
trace10=go.Scatter(x=x[[i_ud+1]],y=CDF[[i_ud+1]],mode='markers', hoverinfo="text",
                  textposition="middle left",marker=dict(size=9,symbol="circle")
                  ,name="Successful comparison",marker_color="blue",text="u<cdf, stop <br> N_searches="+str(-int(lmbda)+1+i_ud))
trace11=go.Scatter(x=x[[i_ud]],y=CDF[[i_ud]],mode='markers', hoverinfo="text",
                  textposition="middle left",marker=dict(size=9,symbol="circle")
                  ,name="Successful comparison",marker_color="blue",text="u>=cdf, stop <br> N_searches="+str(int(lmbda)+2-i_ud))


g = go.FigureWidget(data=[trace1,trace2,trace3],
                    layout=go.Layout(
                        title=dict(
                        text=''
                        )
                        ,xaxis = dict(title = "Upward Search", range = [-0.5,0.5],showticklabels=False),
                        yaxis = dict(title = "", range = [-.1,1.1]),
                        hovermode=None,
                        margin={'l': 0, 'r': 0, 't': 0, 'b': 0},width=400, height=400   
                    ))
g_r = go.FigureWidget(data=[trace1,trace2,trace3],
                    layout=go.Layout(
                        title=dict(
                        text=''
                        )
                        ,xaxis = dict(title = "Upward/Downward search", range = [-0.5,0.5],showticklabels=False),
                        yaxis = dict(title = "", range = [-.1,1.1]),
                        hovermode=None,
                        margin={'l': 0, 'r': 0, 't': 0, 'b': 0},width=400, height=400   
                    ))
 

def response(change):
    Probs,CDF,x,Outcomes,RN,lmbda=process_data()
    with g.batch_update():
        g.data[0].x = x
        g.data[0].y = Probs
        g.data[0].marker.color=CDF
        g.data[0].text=Outcomes
        g.data[1].y=[RN]
        i=convert(Outcomes,CDF,RN)  
        i_ud,downward=convert_updown(lmbda,RN)   
        g.data[1].text="u="+str(RN)+"<br>converts to "+str(Outcomes[i])
        g.data[2].x = x
        g.data[2].y =CDF
        g.data[2].text=np.round(CDF,2)
        
        g_r.data[0].x = x
        g_r.data[0].y = Probs
        g_r.data[0].marker.color=CDF
        g_r.data[0].text=Outcomes
        g_r.data[1].y=[RN]
        g_r.data[1].text="u="+str(RN)+"<br>converts to "+str(Outcomes[i])
        g_r.data[2].x = x
        g_r.data[2].y =CDF
        g_r.data[2].text=np.round(CDF,2)
        

        if search_rn.value==True: 
            g.data[3].x=x[1:i+1]
            g.data[3].y=CDF[1:i+1]
            g.data[4].x=x[[i+1]]
            g.data[4].y=CDF[[i+1]]
            g.data[4].text="u<cdf, stop <br> N_searches="+str(int(i)+1)
            g_r.data = [g_r.data[0], g_r.data[1],g_r.data[2]]
            
            if downward==0:
                g_r.add_trace(trace6)
                g_r.add_trace(trace8)
                g_r.add_trace(trace10)
            else: 
                g_r.add_trace(trace7)
                g_r.add_trace(trace9)
                g_r.add_trace(trace11)

            g.data[3].x=x[1:i+1]
            g.data[3].y=CDF[1:i+1]
            g.data[4].x=x[[i+1]]
            g.data[4].y=CDF[[i+1]]
            g_r.data[3].x=x[[int(lmbda)+1]]
            g_r.data[3].y=CDF[[int(lmbda)+1]]
            if downward==0:
                g_r.data[4].x=x[int(lmbda)+2:i_ud+1]
                g_r.data[4].y=CDF[int(lmbda)+2:i_ud+1]
                g_r.data[5].x=x[[i_ud+1]]
                g_r.data[5].y=CDF[[i_ud+1]]
                g_r.data[5].text="u<cdf, stop <br> N_searches="+str(-int(lmbda)+1+i_ud)
 
            else:
                g_r.data[4].x=x[i_ud+1:int(lmbda)+1]
                g_r.data[4].y=CDF[i_ud+1:int(lmbda)+1]
                g_r.data[5].x=x[[i_ud]]
                g_r.data[5].y=CDF[[i_ud]]
                g_r.data[5].text="u>=cdf, stop <br> N_searches="+str(int(lmbda)+2-i_ud)


def response2(change):
    if search_rn.value==True:
        with g.batch_update():
            g.add_trace(trace4)
            g.add_trace(trace5)
            Probs,CDF,x,Outcomes,RN,lmbda=process_data()
            i=convert(Outcomes,CDF,RN)   
            i_ud,downward=convert_updown(lmbda,RN)     
            if downward==0:
                g_r.add_trace(trace6)
                g_r.add_trace(trace8)
                g_r.add_trace(trace10)
            else: 
                g_r.add_trace(trace7)
                g_r.add_trace(trace9)
                g_r.add_trace(trace11)

            g.data[3].x=x[1:i+1]
            g.data[3].y=CDF[1:i+1]
            g.data[4].x=x[[i+1]]
            g.data[4].y=CDF[[i+1]]
            g_r.data[3].x=x[[int(lmbda)+1]]
            g_r.data[3].y=CDF[[int(lmbda)+1]]
            g.data[4].text="u<cdf, stop <br> N_searches="+str(int(i)+1)
            if downward==0:
                g_r.data[4].x=x[int(lmbda)+2:i_ud+1]
                g_r.data[4].y=CDF[int(lmbda)+2:i_ud+1]
                g_r.data[5].x=x[[i_ud+1]]
                g_r.data[5].y=CDF[[i_ud+1]]
                g_r.data[5].text="u<cdf, stop <br> N_searches="+str(-int(lmbda)+1+i_ud)
 
            else:
                g_r.data[4].x=x[i_ud+1:int(lmbda)+1]
                g_r.data[4].y=CDF[i_ud+1:int(lmbda)+1]
                g_r.data[5].x=x[[i_ud]]
                g_r.data[5].y=CDF[[i_ud]]
                g_r.data[5].text="u>=cdf, stop <br> N_searches="+str(int(lmbda)+2-i_ud)

 
 
    else:
        g.data = [g.data[0], g.data[1],g.data[2]]
        g_r.data = [g_r.data[0], g_r.data[1],g_r.data[2]]


def change_random(b):
    RN_text.value=str(np.random.rand())

RNG.on_click(change_random )
lmbda_value.observe(response,names="value"  )
search_rn.observe(response2,names="value")
RN_text.observe(response,names="value")
  
        
container1 = widgets.HBox([RNG,RN_text])
container2 = widgets.HBox([lmbda_value])
container4 = widgets.HBox([search_rn])
container3 = widgets.HBox([FB])
container5 = widgets.HBox([g,g_r ])
 

Widget1=widgets.VBox([container1,container2,container3,container4, container5])
Widget1